In [4]:
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model


import runtime_embedding_resnet
#from xyz import extract_histogram

In [5]:
# Load the ResNet50 model pre-trained on ImageNet (taktes around 2 sec)
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [ ]:
# Load embeddings and histograms from pickle files
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

with open('histograms.pkl', 'rb') as f:
    histograms = pickle.load(f)


#todo

def extract_histogram(image_path):
    # Dummy function, replace with your actual histogram extraction code
    pass

#todo

def find_similar_images(input_img_path, model, embeddings, image_paths, histograms, top_n_embeddings=1000, top_n_histograms=5):
    # Step 1: Find similar images based on embeddings
    input_embedding = runtime_embedding_resnet.extract_embedding(image_path, model)(input_img_path, model)
    if input_embedding is None:
        print(f"Failed to extract embedding for input image: {input_img_path}")
        return []

    similarities = cosine_similarity([input_embedding], embeddings)[0]
    indices = np.argsort(similarities)[::-1][:top_n_embeddings]  # top N closest images based on embeddings
    closest_image_paths = [image_paths[i] for i in indices]

    # Extract histograms for the closest images
    closest_histograms = [histograms[i] for i in indices]

    # Extract histogram for the input image
    input_histogram = extract_histogram(input_img_path)

    # Step 2: Find similar images based on histograms among the closest ones
    if input_histogram is None:
        print(f"Failed to extract histogram for input image: {input_img_path}")
        return []

    histogram_similarities = pairwise_distances([input_histogram], closest_histograms, metric='euclidean')[0]
    histogram_indices = np.argsort(histogram_similarities)[:top_n_histograms]

    similar_images = [(closest_image_paths[i], histogram_similarities[i]) for i in histogram_indices]
    
    return similar_images



input_img_path = 'path/to/query/image.jpg'
similar_images = find_similar_images(input_img_path, model, embeddings, image_paths, histograms)
print(similar_images)
